In [10]:
# -*- coding: utf-8 -*-
# This code was originally developed by Prof. Hiromasa Kaneko and the original version is found at https://github.com/hkaneko1985/python_doe_kspub
# Modified by Masanori Kodera 

import pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.model_selection import KFold, cross_val_predict
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, ConstantKernel, Matern, DotProduct
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

number_of_selecting_samples = 10
regression_method = 'gpr_one_kernel'
acquisition_function = 'EI'
fold_number = 10 
relaxation = 0.01

dataset = pd.read_csv('selected_samples.csv', index_col=0, header=0) #input file
remaining_samples = pd.read_csv('remaining_samples.csv', index_col=0, header=0) # remaining samples's list

y = dataset.iloc[:, 5]
x = dataset.iloc[:, 1:4]

deleting_variables = x.columns[x.std() == 0]
x = x.drop(deleting_variables, axis=1)
x_prediction = remaining_samples.iloc[:, 1:4].drop(deleting_variables, axis=1)
cumulative_variance = np.zeros(x_prediction.shape[0]) 

kernels = ConstantKernel() * RBF() + WhiteKernel() + DotProduct()

# Bayesian optimization
next_samples = pd.DataFrame([], columns=x_prediction.columns)  
for sample_number in range(number_of_selecting_samples):
    autoscaled_y = (y - y.mean()) / y.std()
    autoscaled_x = (x - x.mean()) / x.std()
    autoscaled_x_prediction = (x_prediction - x.mean()) / x.std()
    model = GaussianProcessRegressor(alpha=0, kernel=kernels) 
    model.fit(autoscaled_x, autoscaled_y)  
    
    if sample_number == 0:

        autoscaled_estimated_y, autoscaled_estimated_y_std = model.predict(autoscaled_x, return_std=True)
        estimated_y = autoscaled_estimated_y * y.std() + y.mean() 
        estimated_y_std = autoscaled_estimated_y_std * y.std()  
        estimated_y = pd.DataFrame(estimated_y, index=x.index, columns=['estimated_y'])
        estimated_y_std = pd.DataFrame(estimated_y_std, index=x.index, columns=['std_of_estimated_y'])
        
        y_for_save = pd.DataFrame(y)
        y_for_save.columns = ['actual_y']
        y_error_train = y_for_save.iloc[:, 0] - estimated_y.iloc[:, 0]
        y_error_train = pd.DataFrame(y_error_train)
        y_error_train.columns = ['error_of_y(actual_y-estimated_y)']
        results_train = pd.concat([y_for_save, estimated_y, y_error_train, estimated_y_std], axis=1) 
        results_train.to_csv('estimated_y_in_detail_{0}.csv'.format(regression_method))  
        
        cross_validation = KFold(n_splits=fold_number, random_state=9, shuffle=True) 
        autoscaled_estimated_y_in_cv = cross_val_predict(model, autoscaled_x, autoscaled_y)  
        estimated_y_in_cv = autoscaled_estimated_y_in_cv * y.std() + y.mean()  
        estimated_y_in_cv = pd.DataFrame(estimated_y_in_cv, index=x.index, columns=['estimated_y'])

        y_error_in_cv = y_for_save.iloc[:, 0] - estimated_y_in_cv.iloc[:, 0]
        y_error_in_cv = pd.DataFrame(y_error_in_cv)
        y_error_in_cv.columns = ['error_of_y(actual_y-estimated_y)']
        results_in_cv = pd.concat([y_for_save, estimated_y_in_cv, y_error_in_cv], axis=1) 
        results_in_cv.to_csv('estimated_y_in_cv_in_detail_{0}.csv'.format(regression_method))  
        
    estimated_y_prediction, estimated_y_prediction_std = model.predict(autoscaled_x_prediction, return_std=True)
    estimated_y_prediction = estimated_y_prediction * y.std() + y.mean()
    estimated_y_prediction_std = estimated_y_prediction_std * y.std()

    acquisition_function_prediction = (estimated_y_prediction - max(y) - relaxation * y.std()) * \
                                           norm.cdf((estimated_y_prediction - max(y) - relaxation * y.std()) /
                                                     estimated_y_prediction_std) + \
                                           estimated_y_prediction_std * \
                                           norm.pdf((estimated_y_prediction - max(y) - relaxation * y.std()) /
                                                     estimated_y_prediction_std)

    acquisition_function_prediction[estimated_y_prediction_std <= 0] = 0

    estimated_y_prediction = pd.DataFrame(estimated_y_prediction, x_prediction.index, columns=['estimated_y'])
    estimated_y_prediction_std = pd.DataFrame(estimated_y_prediction_std, x_prediction.index, columns=['std_of_estimated_y'])
    acquisition_function_prediction = pd.DataFrame(acquisition_function_prediction, index=x_prediction.index, columns=['acquisition_function'])

    if sample_number == 0:
        estimated_y_prediction.to_csv('estimated_y_prediction_{0}.csv'.format(regression_method)) 
        estimated_y_prediction_std.to_csv('estimated_y_prediction_{0}_std.csv'.format(regression_method)) 
        acquisition_function_prediction.to_csv('acquisition_function_prediction_{0}_{1}.csv'.format(regression_method, acquisition_function))  

    next_samples = pd.concat([next_samples, x_prediction.loc[acquisition_function_prediction.idxmax()]], axis=0)
    
    x = pd.concat([x, x_prediction.loc[acquisition_function_prediction.idxmax()]], axis=0)
    y = pd.concat([y, estimated_y_prediction.loc[acquisition_function_prediction.idxmax()].iloc[0]], axis=0)
    x_prediction = x_prediction.drop(acquisition_function_prediction.idxmax(), axis=0)
    dataset= pd.concat([dataset, remaining_samples.loc[acquisition_function_prediction.idxmax()]], axis=0)
    remaining_samples = remaining_samples.drop(acquisition_function_prediction.idxmax(), axis=0)
    cumulative_variance = np.delete(cumulative_variance, np.where(acquisition_function_prediction.index == acquisition_function_prediction.iloc[:, 0].idxmax())[0][0])

    next_samples.to_csv('next_samples_bo_{0}_{1}.csv'.format(regression_method, acquisition_function)) 
    remaining_samples.to_csv('new_remaining_samples.csv')
    dataset.to_csv('new_selected_samples.csv')